# Transform UNIONS multi-band catalogue
- Extract ra and dec for clustering redshifts.
- Write bands in gazpar format (https://gazpar.lam.fr)

In [10]:
%matplotlib inline
%config Completer.use_jedi = False

import numpy as np

from astropy.io import ascii, fits
from astropy.table import Table

In [4]:
# Load the multi-band catalogue, matched with spectroscopic DEEP2+3 measurements
multi_band_v2_data_base = './multi-band_UNIONS/w3.ugriz.spec'

data = ascii.read(f'{multi_band_v2_data_base}.txt')

In [5]:
print(data[0:5])

     RA        Dec        CFIS_ID      u    u_err   g    g_err   r    r_err   i    i_err   z    z_err   sg  z_spec zero
----------- ---------- ------------- ------ ----- ------ ----- ------ ----- ------ ----- ------ ----- ----- ------ ----
214.5213675 52.7520212 2258286007117 21.542 0.029  -99.0 -99.0 20.404 0.008 20.412 0.007 20.467 0.011 2.265 0.2444  0.0
214.5722966 52.7502435 2258286007145  -99.0 -99.0  -99.0 -99.0  24.01 0.131 23.632 0.097 23.062  0.11 1.751 1.4011  0.0
214.4421597 52.7506248 2258286007149  -99.0 -99.0 24.991 0.055 24.491 0.146 23.951 0.132 23.948 0.178 1.206 0.6853  0.0
 214.416452 52.7510622 2258286007153  -99.0 -99.0  -99.0 -99.0 23.897  0.11 23.382 0.074 23.038 0.089 1.611 1.1246  0.0
214.4705147 52.7509707 2258286007155  -99.0 -99.0  -99.0 -99.0 24.098 0.143  -99.0 -99.0  -99.0 -99.0 1.668 0.6144  0.0


In [50]:
print(data.dtype)

[('RA', '<f8'), ('Dec', '<f8'), ('CFIS_ID', '<i8'), ('u', '<f8'), ('u_err', '<f8'), ('g', '<f8'), ('g_err', '<f8'), ('r', '<f8'), ('r_err', '<f8'), ('i', '<f8'), ('i_err', '<f8'), ('z', '<f8'), ('z_err', '<f8'), ('sg', '<f8'), ('z_spec', '<f8'), ('zero', '<f8')]


## Write coordinates

In [6]:
primary_hdu = fits.PrimaryHDU()

col_ra = fits.Column(array=data['RA'], name='RA', format='D')
col_dec = fits.Column(array=data['Dec'], name='Dec', format='D')

secondary_hdu = fits.BinTableHDU.from_columns([col_ra, col_dec])

hdu_list_out = fits.HDUList(hdus=[primary_hdu, secondary_hdu])
hdu_list_out.writeto(f'{multi_band_v2_data_base}_radec.fits', overwrite=True)

## Write gazpar output format

Catalogue format

Your catalogue must be an ASCII file containing the following columns in this order:

[id] [band1] [band1_err] [band2] [band2_err] ... [redshift] [z_flag] [ra] [dec] [mask_flag]

    If the column names are present, the line must be commented out with #;

    The identifiers (first column) must be integers;

    Missing values must be indicated with -9999.;

    Upper limits must be indicated with the value of the upper limit in the flux / magnitude column and a negative number for the error (note that the tools deals differently with upper limits);

    Each flux must be associated to a valid error.

### Mark missing data

In [19]:
bands = ['u', 'u_err', 'g', 'g_err', 'r', 'r_err', 'i', 'i_err', 'z', 'z_err']
for band in bands:
    missing = (data[band] == -99)
    data[band][missing] = -9999

In [20]:
print(data[0:5])

     RA        Dec        CFIS_ID       u     u_err     g     g_err    r    r_err    i     i_err     z     z_err    sg  z_spec zero
----------- ---------- ------------- ------- ------- ------- ------- ------ ----- ------- ------- ------- ------- ----- ------ ----
214.5213675 52.7520212 2258286007117  21.542   0.029 -9999.0 -9999.0 20.404 0.008  20.412   0.007  20.467   0.011 2.265 0.2444  0.0
214.5722966 52.7502435 2258286007145 -9999.0 -9999.0 -9999.0 -9999.0  24.01 0.131  23.632   0.097  23.062    0.11 1.751 1.4011  0.0
214.4421597 52.7506248 2258286007149 -9999.0 -9999.0  24.991   0.055 24.491 0.146  23.951   0.132  23.948   0.178 1.206 0.6853  0.0
 214.416452 52.7510622 2258286007153 -9999.0 -9999.0 -9999.0 -9999.0 23.897  0.11  23.382   0.074  23.038   0.089 1.611 1.1246  0.0
214.4705147 52.7509707 2258286007155 -9999.0 -9999.0 -9999.0 -9999.0 24.098 0.143 -9999.0 -9999.0 -9999.0 -9999.0 1.668 0.6144  0.0


In [54]:
keys = ['CFIS_ID', 'u', 'u_err', 'g', 'g_err', 'r', 'r_err', 'i', 'i_err', 'z', 'z_err',
        'z_spec', 'z_flag', 'RA', 'Dec', 'mask_flag']
cols = []

for key in keys:
    if key in ['z_flag', 'mask_flag']:
        dat = np.zeros_like(data['CFIS_ID'])
    elif key == 'CFIS_ID':
        dat = data[key] % 100000
    else:
        dat = data[key]
    cols.append(dat)

t = Table(cols, names=keys)
with open(f'{multi_band_v2_data_base}_gazpar.txt', 'w') as f:
    ascii.write(t, f, Writer=ascii.CommentedHeader, delimiter=' ')